<a href="https://colab.research.google.com/github/psriraj17/MachineTranslation/blob/main/MT_anythinggoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/'MyDrive'

Mounted at /content/drive
/content/drive/MyDrive


In [ ]:
import collections
from tqdm import tqdm
import helper
import numpy as np
#import project_tests as tests
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [11]:
def load_data(file):
    print("Loading data from file {}...".format(file))
    file = open(file, 'r')
    file = file.read()
    all_data = []
    cur_sent = []
    for line in tqdm(file.split('\n')):
        if line == '<s>':
            cur_sent = ''
            continue
        if line in '()':
            continue
        if line == '</s>':
            all_data.append(cur_sent)
            continue
        else:
            cur_sent += line.lower()
            cur_sent += ' '

    return all_data


#source_data = helper.load_data('/content/drive/MyDrive/Colab Notebooks/train-05/train-source.txt')
source_sentences = load_data("/content/drive/MyDrive/Colab Notebooks/train-05/train-source.txt")
target_sentences = load_data("/content/drive/MyDrive/Colab Notebooks/train-05/train-target.txt")
#taregt_data = helper.load_data('/content/drive/MyDrive/Colab Notebooks/train-05/train-target.txt')


print('\nDataset Loaded',source_sentences[0:10])

Loading data from file /content/drive/MyDrive/Colab Notebooks/train-05/train-source.txt...


100%|██████████| 925535/925535 [00:00<00:00, 2204819.60it/s]


Loading data from file /content/drive/MyDrive/Colab Notebooks/train-05/train-target.txt...


100%|██████████| 910805/910805 [00:00<00:00, 2161934.92it/s]


Dataset Loaded ['cinnte go leór , thiocfadh dóbhtha bás a fhagháil ar imeall an phuill udaí . ', 'bhí sé follasach go rabh an poll sin ag foscladh ar an fhairrge ar dhóigh éigin , ná líonadh agus thráigheadh an t-uisce ann . ', "d'fhéadfadh siad bás ' fhagháil ar a bhruach agus na cuirp imtheacht ar an lán mhara amach fríd an phluais . ", 'thiocfadh dóbhtha fosta lámh a chur ina mbás féin , a ghabháil de léim isteach sa pholl ghalach a bhí i n-iarthar an dara taibhlidh - poll mar bhéadh coire de uisce ghalach ann . ', "' na dhiaidh sin , bhí rud éigin do-chreidte agus leamh in gach teóir de'n bheirt . ", 'nuair a scríobh siad an litir sin bhí siad araon i sláinte mhaith agus tréan bidh aca . ', 'bhí fhios againn i gceart nach rabh díothbhail ar bith ortha , na diomaoite de chuid bonnóg chathail tháinig muid ar an áit taiscthe a bhí aca - scealpach bheag eachar dhá charraic - agus arán agus spólaí de chaoir-fheóil shaillte ann . ', "b'fhuras a fheiceáil cá dtáinig an chaoir-fheóil . ",

In [ ]:
source_words_counter = collections.Counter([word for sentence in source_sentences for word in sentence])
target_words_counter = collections.Counter([word for sentence in target_sentences for word in sentence])

print('{} Source words.'.format(len([word for sentence in source_sentences for word in sentence])))
print('{} unique Source words.'.format(len(source_words_counter)))
print('10 Most common words in the Source dataset:')
print('"' + '" "'.join(list(zip(*source_words_counter.most_common(10)))[0]) + '"')
print()
print('{} Target words.'.format(len([word for sentence in target_sentences for word in sentence])))
print('{} unique Target words.'.format(len(target_words_counter)))
print('10 Most common words in the Target dataset:')                
print('"' + '" "'.join(list(zip(*target_words_counter.most_common(10)))[0]) + '"')

4127703 Source words.
75 unique Source words.
10 Most common words in the Source dataset:
" " "a" "h" "i" "n" "r" "s" "e" "d" "g"

3939531 Target words.
62 unique Target words.
10 Most common words in the Target dataset:
" " "a" "i" "h" "n" "r" "s" "e" "c" "t"


In [ ]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    x_tk = Tokenizer(char_level = False)
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


In [ ]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen = length, padding = 'post')

'''tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))
    '''

"tests.test_pad(pad)\n\n# Pad Tokenized output\ntest_pad = pad(text_tokenized)\nfor sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):\n    print('Sequence {} in x'.format(sample_i + 1))\n    print('  Input:  {}'.format(np.array(token_sent)))\n    print('  Output: {}'.format(pad_sent))\n    "

In [ ]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_source_sentences, preproc_target_sentences, source_tokenizer, target_tokenizer =\
    preprocess(source_sentences, target_sentences)
    
max_source_sequence_length = preproc_source_sentences.shape[1]
max_target_sequence_length = preproc_target_sentences.shape[1]
source_vocab_size = len(source_tokenizer.word_index)
target_vocab_size = len(target_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_source_sequence_length)
print("Max French sentence length:", max_target_sequence_length)
print("English vocabulary size:", source_vocab_size)
print("French vocabulary size:", target_vocab_size)

Data Preprocessed
Max English sentence length: 215
Max French sentence length: 213
English vocabulary size: 27496
French vocabulary size: 24967


In [ ]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = ''

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [ ]:
def model_final(input_shape, output_sequence_length, source_vocab_size, target_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    model = keras.models.Sequential()
    model.add(Embedding(input_dim=source_vocab_size,output_dim=128,input_length=input_shape[1]))
    model.add(Bidirectional(GRU(256,return_sequences=False)))
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(GRU(256,return_sequences=True)))
    model.add(Dense(target_vocab_size,activation='softmax'))
    learning_rate = 0.005
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model
#tests.test_model_final(model_final)


print('Final Model Loaded')
# TODO: Train the final model

Final Model Loaded


In [18]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    tmp_X = pad(preproc_source_sentences)
    model = model_final(tmp_X.shape,
                        preproc_target_sentences.shape[1],
                        len(source_tokenizer.word_index)+1,
                        len(target_tokenizer.word_index)+1)
    
    model.fit(tmp_X, preproc_target_sentences, batch_size = 100, epochs = 10, validation_split = 0.2)
 
    return model

In [ ]:
model = final_predictions(preproc_source_sentences, preproc_target_sentences, source_tokenizer, target_tokenizer)

Epoch 1/10
362/362 [==============================] - 295s 802ms/step - loss: 0.6822 - accuracy: 0.9220 - val_loss: 0.5165 - val_accuracy: 0.9336
Epoch 2/10
 14/362 [>.............................] - ETA: 4:08 - loss: 0.5936 - accuracy: 0.9245

In [14]:
## DON'T EDIT ANYTHING BELOW THIS LINE
    

y_id_to_word = {value: key for key, value in target_tokenizer.word_index.items()}
y_id_to_word[0] = ''

sentence = 'cinnte go leór , thiocfadh dóbhtha bás a fhagháil ar imeall an phuill udaí .'
sentence = [source_tokenizer.word_index[word] for word in sentence.split()]
sentence = pad_sequences([sentence], maxlen=preproc_source_sentences.shape[-1], padding='post')
sentences = np.array([sentence[0], preproc_source_sentences[0]])
predictions = model.predict(sentences, len(sentences))

print('Sample 1:')
print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    #print('Il a vu un vieux camion jaune')
print('Sample 2:')
print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
print(' '.join([y_id_to_word[np.max(x)] for x in preproc_target_sentences[0]]))




IndentationError: ignored